# 1. Install and Import dependencies

To install:

`pip install 'git+https://github.com/DLR-RM/stable-baselines3#egg=stable-baselines3[extra]'`

The version on PyPI is currently broken, installing directly from GitHub fixes this.

You may also want to install PyTorch for CUDA, if available:

[PyTorch Install Guide](https://pytorch.org/get-started/locally/)

In [1]:
import os
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# 2. Load Environment

[CartPole](https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py)

In [2]:
env_name = 'CartPole-v0'
env = gym.make(env_name)

## Environment Example

In [3]:
episodes = 5
for episode in range(episodes):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print(f'Episode {episode+1} finished with score {score}')
env.close()

Episode 1 finished with score 13.0
Episode 2 finished with score 10.0
Episode 3 finished with score 26.0
Episode 4 finished with score 13.0
Episode 5 finished with score 15.0


## Environment Spaces

[Wiki](https://stable-baselines3.readthedocs.io/en/master/guide/algos.html)

![RL Algorithm Comparison](./RL_Alg_Comparison.png)

In [4]:
print(env.observation_space)
print(env.action_space)

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)
Discrete(2)


# 3. Training

In [5]:
log_path = os.path.join('Training', 'Logs')

In [6]:
env = gym.make(env_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cuda device


At this point I got a big long error due to having CUDA 11.4 and PyTorch for CUDA 11.3. I downgraded to CUDA 11.3 using [this guide](https://denishartl.com/installing-cuda-11-3-cudnn-tensorflow-2-4-jupyter-on-a-headless-ubuntu-20-04-server/)

In [7]:
model.learn(total_timesteps=20000)

Logging to Training/Logs/PPO_12
-----------------------------
| time/              |      |
|    fps             | 1143 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 885         |
|    iterations           | 2           |
|    time_elapsed         | 4           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008366071 |
|    clip_fraction        | 0.104       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.687      |
|    explained_variance   | -0.00159    |
|    learning_rate        | 0.0003      |
|    loss                 | 6           |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0151     |
|    value_loss           | 54.8        |
-----------------------------------------
--

# 4. Save and Reload Model

In [8]:
PPO_Path = os.path.join('Training', 'Saved_Models', 'PPO_Model_Cartpole')
model.save(PPO_Path)

In [9]:
del model
model = PPO.load(PPO_Path, env=env)

# 5. Evaluation

In [10]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

/home/stark/anaconda3/envs/RL/lib/python3.10/site-packages/stable_baselines3/common/evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(200.0, 0.0)

In [11]:
env.close()

# 6. Test

In [16]:
episodes = 5
for episode in range(episodes):
    obs = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score += reward
    print(f'Episode {episode+1} finished with score {score}')
env.close()

Episode 1 finished with score [200.]
Episode 2 finished with score [128.]
Episode 3 finished with score [200.]
Episode 4 finished with score [200.]
Episode 5 finished with score [200.]
